In [13]:
import json
import os
import re
import pandas as pd
from collections import Counter, defaultdict

In [22]:
def read_json_file(file_path):
    with open(file_path, encoding="utf8") as f:
        data = json.load(f)
    return data

data = read_json_file('twitter-data-small.json')

In [78]:
def filter_tweets_by_keywords(tweets):
    filtered_tweets = []
    for tweet in tweets:
        if any(keyword in tweet['data']['text'].lower() for keyword in ['train', 'bus', 'tram', 'station']):
            filtered_tweets.append(tweet)
    return filtered_tweets

def filter_tweets_by_location(tweets):
    filtered_tweets = []
    for tweet in tweets:
        if tweet['location'] is not None:
            filtered_tweets.append(tweet)
    return filtered_tweets

filtered_tweets = filter_tweets_by_keywords(data)

#filtered_tweets


In [95]:
#sentiment
import nltk
#nltk.download('vader_lexicon')
#nltk.download('punkt')
from nltk.tokenize import word_tokenize
from nltk.sentiment.vader import SentimentIntensityAnalyzer
def get_sentiment_score(tweet):
    analyzer = SentimentIntensityAnalyzer()
    tokens = word_tokenize(tweet)
    scores = analyzer.polarity_scores(tweet)
    sentiment_score = scores['compound']
    return sentiment_score

def get_sentiment_list(filtered_tweets):
    sentiment = []
    for i in range(len(filtered_tweets)):
        filtered_tweets[1]['data']['sentiment'] = get_sentiment_score(filtered_tweets[i]['data']['text'])
        #sentiment.append(get_sentiment_score(filtered_tweets[i]['data']['text']))
    return sentiment
    
sentiment_list = get_sentiment_list(filtered_tweets)
#sentiment_list
filtered_tweets

[{'_id': '1412189492958556162',
  '_rev': '2-dcc234a7b8b8227faf2c83907d85e412',
  'data': {'author_id': '758965228050616320',
   'conversation_id': '1412189492958556162',
   'created_at': '2021-07-05T23:19:46.000Z',
   'entities': {'hashtags': [{'start': 76,
      'end': 92,
      'tag': 'AuspolSoCorrupt'}],
    'urls': [{'start': 93,
      'end': 116,
      'url': 'https://t.co/1yBy8qgpQ9',
      'expanded_url': 'https://twitter.com/slpng_giants_oz/status/1412174233107501059',
      'display_url': 'twitter.com/slpng_giants_o…'}]},
   'geo': {'place_id': '0073b76548e5984f'},
   'lang': 'en',
   'public_metrics': {'retweet_count': 0,
    'reply_count': 0,
    'like_count': 0,
    'quote_count': 0},
   'text': 'It just means “we’ve done something crooked and it’s none of your business”\n#AuspolSoCorrupt https://t.co/1yBy8qgpQ9',
   'sentiment': 0},
  'includes': {'places': [{'full_name': 'Sydney, New South Wales',
     'geo': {'type': 'Feature',
      'bbox': [150.520928608, -34.11834700

In [88]:
#if you want to categorise in positive/negative
def sentiment_category(sentiment_list):
    result = []
    for sen in sentiment_list:
        if sen <= -0.4:
            result.append("negative")
        elif sen >= 0.4:
            result.append("positive")
        else:
            result.append("neutral")
    return result

sentiment_category = sentiment_category(sentiment_list)
#sentiment_category

In [93]:
#map with sentiment

def plot_sentiment_on_map(data):
    vic_map = folium.Map(location=[-37.4713, 144.7852], zoom_start=8)
    for tweet in data:
        if tweet['place']:
            place = tweet['place']
            if place['country_code'] == 'AU' and place['place_type'] == 'city':
                lat = place['bounding_box']['coordinates'][0][0][1]
                lon = place['bounding_box']['coordinates'][0][0][0]
                name = place['name']
                sentiment_score = get_sentiment_score(tweet['text'])
                if sentiment_score > 0.5:
                    color = 'green'
                elif sentiment_score > 0:
                    color = 'lightgreen'
                elif sentiment_score > -0.5:
                    color = 'yellow'
                else:
                    color = 'red'
                folium.CircleMarker([lat, lon], radius=5, color=color, fill=True, fill_opacity=0.7, popup=name + ': ' + str(sentiment_score)).add_to(vic_map)
    return vic_map


In [ ]:
def get_toot_sentiment(instance_url, access_token, #toot):
    analyzer = SentimentIntensityAnalyzer()
    #not sure if needed?
    mastodon = Mastodon(
        access_token=access_token,
        api_base_url=instance_url
    )
    for toot in timeline:
        #regex
        clean_toot = re.sub('<[^<]+?>', '', toot['content'])
        tokens = word_tokenize(clean_toot)
        scores = analyzer.polarity_scores(clean_toot)
        sentiment_score = scores['compound']
        return sentiment_score, clean_toot

#map with sentiment
def plot_sentiment_on_map(data):
    vic_map = folium.Map(location=[-37.4713, 144.7852], zoom_start=8)
    for tweet in data:
        if tweet['place']:
            place = tweet['place']
            if place['country_code'] == 'AU' and place['place_type'] == 'city':
                lat = place['bounding_box']['coordinates'][0][0][1]
                lon = place['bounding_box']['coordinates'][0][0][0]
                name = place['name']
                sentiment_score = get_sentiment_score(tweet['text'])
                if sentiment_score > 0.5:
                    color = 'green'
                elif sentiment_score > 0:
                    color = 'lightgreen'
                elif sentiment_score > -0.5:
                    color = 'yellow'
                else:
                    color = 'red'
                folium.CircleMarker([lat, lon], radius=5, color=color, fill=True, fill_opacity=0.7, popup=name + ': ' + str(sentiment_score)).add_to(vic_map)
    return vic_map

#compare twitter and mastadon sentiment based on location
def compare_sentiment_location():
    twitter_scores_by_location = {}
    toot_scores_by_location = {}
    #twitter
    twitter_tweets = get_twitter_sentiment_location(twitter_api_key, twitter_api_secret_key, twitter_access_token, twitter_access_token_secret)
    for tweet in twitter_tweets:
        location = tweet['location']
        sentiment = tweet['sentiment']
        if location in twitter_scores_by_location:
            twitter_scores_by_location[location].append(sentiment)
        else:
            twitter_scores_by_location[location] = [sentiment]
    #toots
    toot_statuses = get_toot_sentiment_location(instance_url, access_token)
    for status in toot_statuses:
        location = status['location']
        sentiment = status['sentiment']
        if location in toot_scores_by_location:
            toot_scores_by_location[location].append(sentiment)
        else:
            toot_scores_by_location[location] = [sentiment]
    
    #mean for each
    twitter_means_by_location = {}
    for location, scores in twitter_scores_by_location.items():
        mean_score = statistics.mean(scores)
        twitter_means_by_location[location] = mean_score
        
    toot_means_by_location = {}
    for location, scores in toot_scores_by_location.items():
        mean_score = statistics.mean(scores)
        toot_means_by_location[location] = mean_score
    
    #comparison
    for location in set(twitter_means_by_location.keys()) | set(toot_means_by_location.keys()):
        twitter_mean = twitter_means_by_location.get(location, 0)
        toot_mean = toot_means_by_location.get(location, 0)
        if twitter_mean > toot_mean:
            print(f"{location}: Twitter has a higher mean sentiment score than Mastodon.")
        elif toot_mean > twitter_mean:
            print(f"{location}: Mastodon has a higher mean sentiment score than Twitter.")
        else:
            print(f"{location}: The mean sentiment scores for Twitter and Mastodon are the same.")
                       
def process_data(twitter_data_iter, sal_data):
    authors = []
    city_tweets = defaultdict(int)
    unique_city_tweets = defaultdict(lambda: defaultdict(int))

    for tweet in twitter_data_iter:
        author_id = tweet['data']['author_id']
        authors.append(author_id)
        
        full_name = tweet['includes']['places'][0]['full_name']
        city = full_name.split(',')[0].strip().lower()
        
        if city in sal_data:
            city_code = sal_data[city]['gcc']
            city_tweets[city_code] += 1
            unique_city_tweets[author_id][city_code] += 1

    return city_tweets, unique_city_tweets

#for scaling
def get_file_chunks(filename, num_chunks):
    file_size = os.path.getsize(filename)
    chunk_size = file_size // num_chunks
    return [(i * chunk_size, (i + 1) * chunk_size) for i in range(num_chunks)]

 

def main():
    comm = MPI.COMM_WORLD
    rank = comm.Get_rank()
    num_procs = comm.Get_size()

    sal_data = read_json_file('sal.json')

    # Calculate the number of lines in the file
    with open('bigTwitter.json', 'r') as f:
        total_lines = sum(1 for _ in f)

    # Calculate the lines each process should read
    lines_per_proc = total_lines // num_procs
    start_line = rank * lines_per_proc
    end_line = start_line + lines_per_proc if rank != num_procs - 1 else total_lines

    twitter_data_iter = read_json_file_iteratively('#######.json')


    # TBC - what we actually want to do, easy with functions

if _name_ == "_main_":
    main()